# Using Special Ops

This notebook dives into some technical details of the ImageJ Ops library.

It is recommended that you first read and understand the <a href="../1-Using-ImageJ/2-ImageJ-Ops.ipynb">ImageJ Ops notebook</a> before tackling this one.

In [1]:
%classpath config resolver scijava.public "https://maven.scijava.org/content/groups/public"
%classpath add mvn net.imagej imagej 2.0.0-rc-71
ij = new net.imagej.ImageJ()
ij.getVersion()

Added new repo: scijava.public


2.0.0-rc-71

This tutorial covers the so-called `SPECIAL` ops. Every special op produces one primary output from some fixed number of primary inputs. The number of such inputs for a given op is known as its `ARITY`. The framework provides interfaces for three arities:
* 1 input = UNARY e.g.: eight = math.sqrt(64)
* 2 inputs = BINARY e.g.: eight = math.add(3, 5)
* 0 inputs = NULLARY e.g.: zero = math.zero()
Additional arities (e.g., ternary) are feasible, but not implemented.

The `SPECIAL` ops can be classified into three main categories: `COMPUTER`, `FUNCTION` and `INPLACE`:
* `COMPUTER`: An op which will write its result to pre-allocated output, i.e. an empty (zero) image
* `FUNCTION`: An op which will allocate on its own and then return it
* `INPLACE`: An op which will write its output to one of the input arguments, hence mutating one of the inputs

Special ops may also have any number of additional secondary inputs, which do not affect the arity of the op. More on that below.

Let's try some example op calls, to illustrate the difference between these various kinds of special ops.

First, we will create some test images to work with. In order to do that, we define and use a helper function, which creates an image of specified width and height using the `create.img` op, and then populates the contents of that image using the `image.equation` op, which sets the value of each sample according to a specified formula. In this case, we create four images depicting different patterns based on the location of a sample, which may be accessed via the variable `p[]` inside the formula.

In [2]:
import net.imglib2.type.numeric.real.DoubleType

// Helper function to create DoubleType images from a formula
imageFromFormula = { name, width, height, formula ->      
    img = ij.op().run("create.img", [width, height], new DoubleType())
    ij.op().run("image.equation", img, formula)
    
    println("Created image: " + name + " of size [" + width + "," + height + "]")
    
    return img
}

sinusoid = imageFromFormula("Sinusoid", 150, 150, "10 * (Math.cos(0.3*p[0]) + Math.sin(0.5*p[1]))")
gradient = imageFromFormula("Gradient", 150, 150, "p[0] + p[1]")
spot     = imageFromFormula("Circle",   150, 150, "Math.sqrt((p[0]-75)*(p[0]-75) + (p[1]-75)*(p[1]-75))")
cross    = imageFromFormula("Diamond",  150, 150, "Math.abs(p[0]-75) + Math.abs(p[1]-75)")

ij.notebook().display([["sinusoid": sinusoid, "gradient": gradient, "spot": spot, "cross": cross]])

Created image: Sinusoid of size [150,150]
Created image: Gradient of size [150,150]
Created image: Circle of size [150,150]
Created image: Diamond of size [150,150]


sinusoid gradient spot cross <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAJYAAACWCAYAAAA8AXHiAAAHW0lEQVR42u3S03aYYRhE4be2bdt2U6Wpbdu27SZpU9u2bdu2bXu68v23MIez19p38JiZIfgQIUIgZMiQCBUqFEKHDo0wYcIgbNiw7nDhwiF8+PCIECECIkaMiEiRIiFy5MjuKFGiIGrUqIgWLRqiR4+OGDFiIGbMmO5YsWIhduzYiBMnDuLGjYt48eIhfvz47gQJEiBhwoRIlCgREidOjCRJkiBp0qTuZMmSIXny5EiRIgVSpkyJVKlSIXXq1O40adIgbdq0SJcuHdKnT48MGTIgY8aM7kyZMiFz5szIkiULsmbNimzZsiF79uzuHDlyIGfOnMiVKxdy586NPHnyIG/evO58+fIhf/78KFCgAAoWLIhChQqhcOHC7iJFiqBo0aLw8fFBsWLFULx4cZQoUQIlS5Z0lypVCr6+vihdujT8/PxQpkwZlC1b1l2uXDmUL18eFSpUQMWKFVGpUiVUrlzZXaVKFVStWhXVqlVD9erVUaNGDdSsWdNdq1Yt1K5dG3Xq1EHdunVRr1491K9f392gQQM0bNgQjRo1QuPGjdGkSRM0bdrU3axZMzRv3hwtWrRAy5Yt0apVK7Ru3drdpk0btG3bFu3atUP79u3RoUMHdOzY0d2pUyd07twZXbp0QdeuXdGtWzd0797d3aNHD/Ts2RO9evVC79690adPH/Tt29fdr18/9O/fHwMGDMDAgQMxaNAgDB48GEOGDAk2JVRCxUXlYAmVULFRDR06FCZUQsVGNWzYMJhQCRUb1fDhw2FCJVRsVCNGjIAJlVCxUY0cORImVELFRjVq1CgPllAJFROVgyVUQsVGNXr0aJhQCRUb1ZgxY2BCJVRsVGPHjoUJlVCxUY0bNw4mVELFRjV+/HiYUAkVG5W/v78HS6iEionKwRIqoWKjCggIgAmVULFRBQYGwoRKqNioJkyYABMqoWKjmjhxIkyohIqNKigoCCZUQsVGNWnSJA+WUAkVE5WDJVRCxUY1efJkmFAJFRvVlClTYEIlVGxUU6dOhQmVULFRTZs2DSZUQsVGNX36dJhQCRUb1YwZMzxYQiVUTFQOllAJFRvVzJkzYUIlVGxUs2bNggmVULFRzZ49GyZUQsVGNWfOHJhQCRUb1dy5c2FCJVRsVPPmzfNgCZVQMVE5WEIlVGxU8+fPhwmVULFRLViwACZUQsVGtXDhQphQCRUb1aJFi2BCJVRsVIsXL4YJlVCxUS1ZssSDJVRCxUTlYAmVULFRLV26FCZUQsVGtWzZMphQCRUb1fLly2FCJVRsVCtWrIAJlVCxUa1cuRImVELFRrVq1SoPllAJFROVgyVUQsVGtXr1aphQCRUb1Zo1a2BCJVRsVGvXroUJlVCxUa1btw4mVELFRrV+/XqYUAkVG9WGDRs8WEIlVExUDpZQCRUb1caNG2FCJVRsVJs2bYIJlVCxUW3evBkmVELFRrVlyxaYUAkVG9XWrVthQiVUbFTbtm3zYAmVUDFROVhCJVRsVNu3b4cJlVCxUe3YsQMmVELFRrVz506YUAkVG9WuXbtgQiVUbFS7d++GCZVQsVHt2bPHgyVUQsVE5WAJlVCxUe3duxcmVELFRrVv3z6YUAkVG9X+/fthQiVUbFQHDhyACZVQsVEdPHgQJlRCxUZ16NAhmFAJFRvV4cOHPVhCJVRMVA6WUAkVG9WRI0dgQiVUbFRHjx6FCZVQsVEdO3YMJlRCxUZ1/PhxmFAJFRvViRMnYEIlVGxUJ0+e9GAJlVAxUTlYQiVUbFSnTp2CCZVQsVGdPn0aJlRCxUZ15swZmFAJFRvV2bNnYUIlVGxU586dgwmVULFRnT9/3oMlVELFROVgCZVQsVFduHABJlRCxUZ18eJFmFAJFRvVpUuXYEIlVGxUly9fhgmVULFRXblyBSZUQsVGdfXqVQ+WUAkVE5WDJVRCxUZ17do1mFAJFRvV9evXYUIlVGxUN27cgAmVULFR3bx5EyZUQsVGdevWLZhQCRUb1e3btz1YQiVUTFQOllAJFRvVnTt3YEIlVGxUd+/ehQmVULFR3bt3DyZUQsVGdf/+fZhQCRUb1YMHD2BCJVRsVA8fPvRgCZVQMVE5WEIlVGxUjx49ggmVULFRPX78GCZUQsVG9eTJE5hQCRUb1dOnT2FCJVRsVM+ePYMJlVCxUT1//tyDJVRCxUTlYAmVULFRvXjxAiZUQsVG9fLlS5hQCRUb1atXr2BCJVRsVK9fv4YJlVCxUb158wYmVELFRvX27VsPllAJFROVgyVUQsVG9e7dO5hQCRUb1fv372FCJVRsVB8+fIAJlVCxUX38+BEmVELFRvXp0yeYUAkVG9Xnz589WEIlVExUDpZQCRUb1ZcvX2BCJVRsVF+/foUJlVCxUX379g0mVELFRvX9+3eYUAkVG9WPHz9gQiVUbFQ/f/70YAmVUDFROVhCJVRsVL9+/YIJlVCxUf3+/RsmVELFRvXnzx+YUAkVG9Xfv39hQiVUbFT//v2DCZVQsVEFZ0IlVGxUwf0HHB3r7g7hHXkAAAAASUVORK5CYII=" /> <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAJYAAACWCAYAAAA8AXHiAAAlpUlEQVR42u2ddYws19XEW+FEIYcZHGZmZmZmZmYGh5mZ2Q4zo5mZmZktW5Yty/19Z6Qa1dbWudC7m/zTkY6ct9Pdb3f696rq1p2dGcb//9+FF144XnDBBeP5558/nnfeeeO55547nnPOOePZZ589nnXWWeOZZ545nn766eOpp546nnLKKeNJJ500nnjiiePxxx8/HnfcceMxxxwzHn300eORRx45HnHEEePhhx8+HnrooeMhhxwyHnTQQeOBBx44HnDAAeN+++037rvvvuM+++wz7r333uOee+457rHHHuPuu+8+7rbbbuOuu+467rLLLuNOO+007rjjjuMOO+wwbr/99uN22203brvttuM222wzbr311uNWW221mC233HL8z3/+M/773/8e//Wvfy3mn//85/iPf/xj/Pvf/76Yv/3tb4v561//upi//OUvi/nzn/+8mD/96U8r5o9//OPk0Wvh78Dfie8B3xO+x/h+4/vGzxA/T/xc8fPhZ42fO37+eB7i+YjnJZ6feJ7i+YrnLZ6/eB7j+YznNZ7feJ7j+Y7nPZ7/uA9xP+K+xP2J+xT3K+5b3L+4j3E/477G/Y37HPc77nvc/+AgeAgugo/gJHgJboKf4Ch4GmpQnXHGGSlUxx577OKbOOqooxbfVHyDhx122OKbPfjggxfffPwg+++//+KHAlB77bXX4odWoHbeeecqUK0wMUgKEUD4wx/+sJzf//73q+Z3v/tdddx5fF2FjmFj0FohKwEWz58CFs9zPN8ALO5D3I+4L3F/4j7F/Yr7Fvcv7mPcz7ivcX8zuIKLElxDK1Qnn3zy4uInnHBCEar4VxDfrFMpABX/mhSo+FfXA1QGEysSg6QAMRy//e1vV8xvfvOb7tFrOPgYNgaNFS2DrBWweB4VMKhXPP9OveJ+xX0rwRX3Pe5/cNAC19ALVUhk/GUhmfGXZ9bnVIptj4GKf21TgGKYWJVYkRQkhefXv/71cn71q1+tml/+8pfpuOP5egqdA40VTdUMkPUCBnsEYGyPTr0ya4z7G/c57nfc9x64hgyq0047bQlVSKFChTyVWV9NpTRH1YBimBQop0oMkkLkoPnFL36xnJ///Ofdw+c7+BxsDJqqmQLGkNUAK+Uvp16ZNXLuYriCB8AVnDi4hh6oNKSXrM9lqZLtOaBK6uSUSRWJQVKAGIqf/exny/npT39qZ4sttlhOdgxfx0HHsDForGiZkmUq5gAr2a

## BINARY OPS
First we will look at binary ops. A `BINARY` op is one with two primary inputs, which produces an output. Suppose we want to add two images together. There are several versions of the `math.add` op. In the following example we will try the functional version:

In [3]:
//Pointwise add FUNCTION
functionOutput = ij.op().run("math.add", sinusoid, gradient)
ij.notebook().display(functionOutput)

<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAJYAAACWCAYAAAA8AXHiAAAzUklEQVR42u2dc/AtSfP057Ft27Zt27Zt27Zt27Zt27af+UXOG58TubndPT0Hd3efd29ExffMuWf+y6jKzs6qGg5+8IOPhzjEIabQ54Me9KDjQQ5ykPFABzrQeMADHnA8wAEOMA7DMIU+63v9v357qEMdajzsYQ87HuEIRxiPfOQjj0c72tHGYxzjGOMxj3nMVej56Ec/+njUox51+s2RjnSk8YhHPOL0jv7qWd/r//XbYx/72OPxj3/88UQnOtF40pOedDz5yU8+nvKUpxxPdapTjac+9amnv3o+xSlOMZ7sZCebfnPiE594+j2hZ97V70972tOOZzjDGcYzn/nM49nOdrbxnOc853iuc51rPM95zrOKc5/73NP35zjHOabfnPWsZx3PcpazTO8o9Fnf6//12/Od73zjhS50ofGiF73oeIlLXGK81KUuNV7mMpcZL3vZy65Cz/r+kpe85Hjxi198vNjFLjb9/iIXucj0V8/6Xr/R769whSuMV77ylcerXe1q4zWvec3xOte5znjd6153vN71rjde//rXn/7qWd9f61rXGq9xjWuMV7/61affE3rWu9e+9rWn39/whjccb3rTm463uMUtxlvf+tbjbW972/H2t7/9eIc73GEKfb7d7W43fa//v9WtbjX99uY3v/kqbnnLW07f6zd65853vvN4t7vdbbznPe853uc+9xnvd7/7jQ94wAPGBz7wgasYDnOYw0zg0F+FwHLIQx5yAs7BDnawCUSEQKfv9P+HPvShx8Mf/vATMBwUxznOccbjHe94qzjucY87fXesYx1rBTIBUO8oAKP+X789wQlOMAEDUJzmNKcZT3e6042nP/3pV6FngQWgCWT6PaFngY93z3SmM01AARTnPe95x/Of//zjBS5wgfGCF7zg9FfPAkuCjNAz7+r3F77whSdgAIrLX/7y4xWveMXxSle60ir0rO8vd7nLTSC79KUvvQKaAjDy7lWvetUJLIDiBje4wXijG91ovPGNb7wKPet7AQ2Q6feEnnlXv7/ZzW42AQNA3elOd5qAcZe73GW8613vOv3Vs74XaADZbW5zm1X4u/r9Pe5xj/He9773BCiB6MEPfvD40Ic+dHzYwx42hT4PZA79FVAOd7jDTUATcAQgsplCz2QpQCVgKDMBKAHjhCc84Sp76LMykP5PvxH4BCIymj7zrn57kpOcZG+ZRsAgcyj0fMYznnECmX4jAAEy/dWzvtf/AyoBQ6ARgAQmAYPMob96VgYCZAKQZzTAqHf1+1KmucpVrrLKHAKJnvW9QKPfCGCe0fSs7wVCQCVgCDQC0E1ucpMJGJ499KzvBRplI0BGVtOzvtdv9PtSprn73e8+gYPQs74XaASeO97xjquMptCz3hUQ9Xtlqfvf//7jgx70oAlED3/4w8dHPvKR46Me9agp9HmgFOmvAoCRxQQkQmDTd/p/gdEzFcAQmAQOQtlH3wtwnsH0jmc4MpXKm7KNg0plSOAgKFGAiwxGkOF418sXoPJS5CVK4NJvyGCEA1K/V8ZRtnFgqAwJHISeARgZTL9XhlLku16+AJXKkjIOoWd9r/JGBgNg+uuA1O+VcZRtHBgqYfe6172mrKO/etb3gIsMRvCuAKjfk6ke8pCHrED16Ec/enzMYx4zhT7PAktgcmB5tuoFlr7bFbD0mxawvAQCrOQ4mwJLWWkpsPSX0pklcA5YykQJLKIELM9WApDAUQOWAFQDVpbAJrCUeQgHFYDycgi3SsJOKVTJ8zJIKXRQUQoJBxelUBxJJU0A8VIIoLwU6jeUQgJulYSdUijwAC6iBCovh6VSKM6kciaAAC4n0g4qAEU51F+4VRJ2SqHAQykEUJmtKIUEvMxLoXiTypmyEeASmACUl0KBKEsh4HLC7qXwEY94xF5KoWIQpyK8/HFKFFkXaYe46/vSadB5lgBGJKAg8ASnSAeXTnSAC64kkJChElCcEgk9824NXAKIgEQ4oCDwOjnqHT9FetbTaRBwUdYEEsIJPKdE/Z7QM+/WwCUACSQCE9yqROAJPet7sh6nQcBF9hFIiASUfguJ119OkS1wibQrexEDJY4MlbLDgQ984EliUOizvq9JDQIIGYhwQHEaPMpRjjK9o9BnToaUVKQGgYSToZe6BBSyA6HnltRABqLEeXYCTGc/+9mn3xN65mQIuJAaBBJOhgIJ4YDiNKjfK9vpLyfDmtTA6c5LnQNKpVOyg35P6Jl3S+DyDETJc0C57EAgU9TAxclQpZEY/NTnEoNA5DqW/tZ0LJVRBwinvdSx9Bt0LAKOlyXVwUUGIlxi0G+cy9V0LGU7lVIHSOpYAlVJx4LjlUpqScei3JGhABRcjtAz7wIulU5xMwcIZS51LAGppmNlSa3pWGhZSAwCknM5OF6CCyJ/3/vedwIXOpYy2KRjCSAEZY8s5eIoIbDp/9GzVDrJWgKPlzfXrchScDkCcJUOAkgPZC5CQNP3ykwOKr3jBwgOAspwnrUEHoFIYCI8UzmonN/VDgLKQAIP5c1BJdCQpeByhJ55Fz3Ls5YyEgKpg0rf6/9F+OFyhJ5LBwH4lsBDeSP0rO8FHgeV8zveRc9KgdTBpb//32QsAWv/jLUHMxb8yk+BCTD4VY1jCTACzxzHcoARqO+8m6BywVOEvaa6O8/SZ7KccyyBykm4wEGUVHd4lv6ivvOug6rFsVDdnWMRZLnkWIAKEi5wuF7lpL3GsXjXQVXiWK66O8DgWfpLlkuOBagEpr1xLMkLqbrnyZBActD/67cpkur0J2CgvtekBjIZ2Y13U8sSoHQCFD9SSSJcv3LV3e8UkRzgVl7CpK4LGKjvqO6lkyEBoPw6x0sYJ0JKEqq7Sw1kMcKvgshUXsJ0AkRyIEpSA5nM1Xe/zvESxolQGYdw/Qoi76USMOr0qN+nlqUTIZKDQp8Hzx4IpSmSEnoWoPwk6Hd8cBuyBhnERVKXIBSIo5mlHBRkDULPKZIiQXCfqO/9JOh3fHAbzx4IpQCNO0TCFXuXGRwU4jZkDThQSSQl9Mw1EOXLQQG3UdYg/JI4JQiXJlJmABQIo+JGhJ5TJNXvCVR3PwkqQwlQiKOPfexjx8c97nFT6PPgbgSXBNKJANFOJ0ISbbKGE+2SEwGSredeop36UoKs5UTIOz5KUI8TYR2iTRZBEnAnAiQbMNaIdupI6URwLkT0EG13IuizgKLvE2QEYEzFXUKoQPT4xz9+fOITnzg+6UlPWsWwqeaEoOluhHU1pwSVOwpKmhN8qEdzQtB0N0Kv5kR283d3pTkJHE6UlW3gREs0pxKoapqTnvW9AwyQ6a+4VAlUyk5PeMITxic/+cnjU57ylPGpT33qKgZKkt/hZfZyiwskndObcyLUbkoSRNvBBdEmyHJZ/iDagMrLkrsPyF6QbYg2JJ3Tm3Mi1G4vSw6utLcgZJbKn8Ah4PglcF4OZ/YiyHJ+enNOhFLuZQke5DIBZBtQlcqfwKHsJKItcEgtJ/TMaQ5w6fcOKt7lCkd3gspUyk4C1dOe9rQpnv70p09/p4zloIJcr3OC84y1jROcn8LyBCdgbeME56By39Q2TnCKnhNcyZC3J05wgErP+l7gcVARvEvGErDEq5SxVAKVsZSlANeUsfxCuMWzcD9QDktZC4BwKvPTWgqaBGSfd0tZC34lMNWuXtCd0JwcmJm1AIiARHgZdDLvLggHZmYt15zytJY8i9CzAzOzFgBR9vHTWknQJNCceDezFgARkIgsg+56gF85MD1rAS5lLgGMGPxC2AGV5j8FPqyWoIlSTiSgkCAIV8rhWnkiJANl2QNMSBAEVps5QdMvhEsKOTIE0gQEHq7l4ILA+4WwA8rdDgRWm5agiYPTnZ0OKCQIwk+T8DQHlyvlWfYAExKEmwJrrgYyl8qi+BYxQM

The above invocation calls a `math.add` `FUNCTION`, which allocates and returns a newly created output image, without mutating any of the input data. This is great from a functional standpoint, but costs space and time. Now suppose we already have a properly dimensioned output image allocated and ready to go. In that case, it would be wasteful to call math.add as a function, only to then copy the data from the newly allocated result back into our already-existing image. Much better would be to tell `math.add` to use our existing output buffer directly. Fortunately, this approach is made possible by `COMPUTER` ops, which require you to specify a pre-allocated output buffer where the result will be stored. 

For example, imagine you want to do some analysis on a large set of images which would require to run e.g. a Gaussian smoothing as an intermediate operation. In this case, this intermediate step would not have to be store as it is not part of the end result. Thus, we can make use of the `COMPUTER` paradigm. We pre-allocate a single output buffer to store the results of the smoothing and continue with our analysis for the current image. For the next image, we simply overwrite the same buffer and continue. This approach is much faster compared to calculating the smoothing using a `FUNCTION` which would create a new output buffer for every intermediate step.

In the following example, we will first pre-allocate an output image and then write the result of the addition to this image:


In [4]:
// Perallocate output image
computerOutput = ij.op().run("create.img", sinusoid)
// Pointwise add using a COMPUTER
ij.op()run("math.add", computerOutput, spot, gradient)
ij.notebook().display(computerOutput)

<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAJYAAACWCAYAAAA8AXHiAAAahklEQVR42u3dg5NtxxYG8J0X27b5Ytu2bdu2bdu2k5f3Ytu2jUoqqaSS9apPqueu6bvY3efcmcncqv0P3POrtb79dc85TdM0gJ/BBhus2/Ovf/2r6xl88MG7PUMMMUTXM+SQQ3Y9Qw01VOsZeuihu55hhhmm9Qw77LBdz3DDDdd6hh9++NYzwggjdD0jjjhi6xlppJFaz8gjj9x6RhllFBh11FFbz2ijjdZ6Rh999NYzxhhjwJhjjtl6xhprrNYz9thjwzjjjNN6xh13XBhvvPFaz/jjjw8TTDABTDjhhK1nookmgoknnhgmmWSS1jPppJPCZJNNBpNPPjlMMcUUMOWUU7aeqaaaCqaeemqYZpppYNppp4XpppsOpp9+ephhhhng3//+N8w444ww00wzwcwzzwyzzDILzDrrrDDbbLPB7LPPDnPMMQfMOeecMNdcc8Hcc88N88wzD8w777ww33zzwfzzzw8LLLAALLTQQrDwwgvDIossAosuuigstthisPjii8MSSywBSy21FCy99NKwzDLLwLLLLgvLL788rLDCCrDiiivCSiutBKussgqsuuqqsNpqq8Hqq68Oa665Jqy11lqw9tprw7rrrgvrrbcerL/++rDhhhvCRhttBBtvvDFsuummsNlmm8Hmm28OW265JWy11VawzTbbwLbbbgvbbbcd7LDDDrDjjjvCzjvvDLvssgvsuuuusPvuu8Mee+wBe+21F+y9996w7777wn777QcHHHAAHHjggXDwwQcHSzZUKSwNVQmsFFWEFVD9U2EtueSSLlhrrLGGC9YWW2wxEKztt99+IFi77bZbC9aee+5pg1UDlTatMKyIygILT6t+WH/DWm655brBWnnllVlY66yzThesDTbYoAvWJptsMhCsrbfeeiBYO+20Ewtrn332acHaf//9W7AOOuigAbA8qNo1rdoBK6CiYAVU/bAGwAprMIUV1mAKK6zBACusQRVWp1DVylccrICqt8NacMEFSVghX3lghXwVYYU1GGGFNRhhhTWYwgprMIUV1iAHK6zBACuswQArrMEA65BDDukOy4OqBJZlDdaGFVD1FVghX2FYIV/lworBHcOKwR3DSoO7GVanUHnXoAYLr8F+WKt0vRFKsGJwx7BicMewYnCnYEnBvQtWLVS508qSr1JYVL7qzbBCvqJgxaohB1Z8Iwyw4hshhuWtGmJwN8EqQVUyrWrmq38SrFg1RFixarDCsnZYlqqBC+6HHnpod1i1UJWuwVJYuGrozbBwOWqBlVuOWqoGzxthN1glqEqmVUm+6iuwYtXghYXLUS8sqcOyVA1ScG/BSkHVROWZVv2wdFi4dU9htaPD0oI7B+uwww4bGFYJqhrTyhLcPa17b4WVnhPWhCV1WJaqQQvu3WCloGqi8oR2S77qS7Bw6+6BZTknLOmwSoJ7F6zaqLzTquYalM4JezMs6QDaU45KHZalajDD0kB1ElXJGiyBFVBhWAHVoISlXZnxHud4O6zS4H744YcPgGUB5UXVD0uGlR5Ac7Ckc8LSDqv2G2GYVl2wehoqag32w8qDlVs1lAT3FqwcUCWoSqdVTutO3cXqqbC0KzOW4xyqw5Kqhtw3Qi5fHXHEEd1htQNVybSS1mBvh0XdxaJgSeeEWjmaWzWUBPcwrbpg5YIqQdWJNajdHu2LsKgOS6oarGeEnuDegmUBVQNVjWnV12FpV2Y85Whu1ZAT3NN8deSRRw6AVQKqBJVnWvUVWNZrydZzwtyqoR3BPUyrFiwPqBqovNMqJ1/1RVi55Wi73gil4E7CKgFVgipnDVph4aqhp8LyHEBrsCxVQ83gnuaro446agAsDyjLlNJQeaeVtAZ7EyzrtWQLLGuHVfpG6ClGw7RqwSoF1WlUljUolaO9EZanHLVUDTWCO1eMBlhHH310d1gcphJQGqrcaeXNVz0ZlucAmoIldVjeN0JPcOfyVQuWF9OgRNXbYVmvJVtad6nDkqqGnODuKUbDGmRheTBZQXlQedagFxZu3fsCLKnDakdwl4rRmK+OOeaYAbC804kDlYOqxrTqrbCsB9DWDku61eCBlVuMhmnVguXFVArKgypnWvVUWNZryZ7W3VI11AjuUjFK5atjjz22OywJkxdUDirvtJLWoOUAelDD8pwTSh2W5VZDaXD35KsWrFxMpaA8qGqtQelack+DldNhaW+EnuAuFaNaviJhSZAkTBwoy5SyrkBuWvV0WNZryRosqWqQbjVQsEqLUSlfHXfccdBokDRMHlClqLRpVQIroOokLOmcUOuwLFWDJ7jXyFd4DbKwNEgSphJQ1hWoTau+CEuqGrQ3Qk9wtx48U+eDcQ0ef/zxf8MqhcRh6iQqalr1BFiWa8nccQ7VYVmqBktw9+Yry/lgXIMtWLmQcjBZQUmoctag1LoPCljSOaFWjmpvhDnB3ZqvpPtXeA2ecMIJf8OyILJg8oKyoPJOK+sa7E2wPFWDBEsqRq35ijofxPkqTKsWrBJEGqZSUBKq0jXYCViWa8meclR7I/QEd2u+0u5fpTVDgHXiiScOgGVFpEGSMHGgtCmVM616KizpnJCC5akaMCxLMVorX+GaIUyrFqxSRBZMHlBeVNoa7MmwcjqsnOBeO19JNUOAddJJJ/0Ny4rHikjDxIHSgnrJtLIc59SEZbmWrMHyVg25xaiWr7RjnHQNtmDVQqRBkjBZppSGyroGByUs6QDa0mF5gnvtfGVZgxHWySefDI0HjgdRLiYLKCsq6xocFLC0crRmcM/JV9oxDlczhGmlwrLisULKAeVF5VmDnYQlnRN6qgYJllSMavmKun/F1QzaGjzllFOgycFjBaRB8oLyoBqUsLRLflqHVSO45+Sr3JohXYMiLA8cDyINEweqBioNVvptfp2EZakaKFhSMarlK+1+e+4aPPXUU6FpNyALJC8oDZU2rToNizsnpDqsnOBuzVfU+SBXM5SswdNOO21gWF4wOYA0SBKoXFSdgqVd8qPKUUvVkFOMUv1Vbs3gWYMtWO2E40GkYaJAaag8a7DTsKSqwRLcrflKu3/VjjV4+umnQ5MLpRSQBRKHqQRVp2Fp5ahWNXjzFdVfcTWDt223rkEzLC+WXEQWTBQoDVWnYGmX/CxVgxTcrfkqt2bIWYPxbDBOqwDrjDPOgKbTaHIgcZgoUFZUnYCllaM5wd3aX3E1QyfW4JlnntkdVgmOGngskCRQGipqWvV0WNZ8xdUM2p95cW177ttgmFYtWJ2CUoJIw0SBklC1A5Z2yY+rGizBPSdfcTVDzhrUrsika/Css86CpiaQEjReSBIoDRW1BqnWvQSWVo5KsHLyVSfWIHVTNE4rvAZZWCUY2oHHAokDJaHiplW7YHFvhFRw1/KVp2aQ1mBOKcp1V3ENnn322dAMKiA1EEmYPKhKYUkH0FKHVZqvaq5B7xUZKrTHNXjOOedA0w4QtcDkQOJApag6DcsS3LV81Yk1aLkpynVXcVqde+653WGVfuA1HwseCyYPqtqwuDdCa76izge5mkFagzlngzndVZxWLVg9AURNQBImClUpLOkA2vJGqOWrmmvQckVG666k0B6mVYB13nnnQdMJCLWg5ELiQHGo8F2sXFie4I7zVbvWYI3uSgrtYVrFNXj++edDU+tD7cSjwfGASlFx08oDi+qwKFg4X2nng9Y1aD0blLorKbRrTXtcg2FaFcPK/aDb9XCANFDUtLLA4g6gLcEd56vSNcidDaZXZHJDu6Vpj9MqwLrgggug6Y0wch4KE4UqFxbVYXH5iluDuGbIWYNSdyWFdunAWWra04ohrsELL7wQmnZ+mJ14JDDak4LCqHJhUcFd66+kNciVounZYG5oz23a04ohrsEwrS666CJoSj6Y3vJQgDRUGiyudefyVekaxGeD6R

In the above call, the same operation is performed, but the output is written into the provided "result" buffer. `COMPUTER` ops do have an important restriction: the output buffer __MUST NOT__ be the same reference as any of the inputs. Otherwise, the computer would end up mutating the input data as a consequence of writing to the output buffer. Hence, computers cannot be used to calculate results "in place" -- that's what `INPLACE` ops are for.


In [5]:
import net.imagej.ops.special.inplace.Inplaces;
import net.imagej.ops.Ops

// Create a copy of one of the example images which we want to mutate
gradientCopy = ij.op().run("copy.rai", gradient)
// Create an INPLACE math.add Op
inplaceOp = Inplaces.binary1(ij.op(), Ops.Math.Add.class, gradientCopy, sinusoid)
""

In [6]:
inplaceOp.mutate(gradientCopy)
max = ij.op()run("stats.max", gradientCopy)
println("The maximum pixel value is: " + max)
ij.notebook().display(gradientCopy)

The maximum pixel value is: 308.6507787063433


<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAJYAAACWCAYAAAA8AXHiAAAzUklEQVR42u2dc/AtSfP057Ft27Zt27Zt27Zt27Zt27af+UXOG58TubndPT0Hd3efd29ExffMuWf+y6jKzs6qGg5+8IOPhzjEIabQ54Me9KDjQQ5ykPFABzrQeMADHnA8wAEOMA7DMIU+63v9v357qEMdajzsYQ87HuEIRxiPfOQjj0c72tHGYxzjGOMxj3nMVej56Ec/+njUox51+s2RjnSk8YhHPOL0jv7qWd/r//XbYx/72OPxj3/88UQnOtF40pOedDz5yU8+nvKUpxxPdapTjac+9amnv3o+xSlOMZ7sZCebfnPiE594+j2hZ97V70972tOOZzjDGcYzn/nM49nOdrbxnOc853iuc51rPM95zrOKc5/73NP35zjHOabfnPWsZx3PcpazTO8o9Fnf6//12/Od73zjhS50ofGiF73oeIlLXGK81KUuNV7mMpcZL3vZy65Cz/r+kpe85Hjxi198vNjFLjb9/iIXucj0V8/6Xr/R769whSuMV77ylcerXe1q4zWvec3xOte5znjd6153vN71rjde//rXn/7qWd9f61rXGq9xjWuMV7/61affE3rWu9e+9rWn39/whjccb3rTm463uMUtxlvf+tbjbW972/H2t7/9eIc73GEKfb7d7W43fa//v9WtbjX99uY3v/kqbnnLW07f6zd65853vvN4t7vdbbznPe853uc+9xnvd7/7jQ94wAPGBz7wgasYDnOYw0zg0F+FwHLIQx5yAs7BDnawCUSEQKfv9P+HPvShx8Mf/vATMBwUxznOccbjHe94qzjucY87fXesYx1rBTIBUO8oAKP+X789wQlOMAEDUJzmNKcZT3e6042nP/3pV6FngQWgCWT6PaFngY93z3SmM01AARTnPe95x/Of//zjBS5wgfGCF7zg9FfPAkuCjNAz7+r3F77whSdgAIrLX/7y4xWveMXxSle60ir0rO8vd7nLTSC79KUvvQKaAjDy7lWvetUJLIDiBje4wXijG91ovPGNb7wKPet7AQ2Q6feEnnlXv7/ZzW42AQNA3elOd5qAcZe73GW8613vOv3Vs74XaADZbW5zm1X4u/r9Pe5xj/He9773BCiB6MEPfvD40Ic+dHzYwx42hT4PZA79FVAOd7jDTUATcAQgsplCz2QpQCVgKDMBKAHjhCc84Sp76LMykP5PvxH4BCIymj7zrn57kpOcZG+ZRsAgcyj0fMYznnECmX4jAAEy/dWzvtf/AyoBQ6ARgAQmAYPMob96VgYCZAKQZzTAqHf1+1KmucpVrrLKHAKJnvW9QKPfCGCe0fSs7wVCQCVgCDQC0E1ucpMJGJ499KzvBRplI0BGVtOzvtdv9PtSprn73e8+gYPQs74XaASeO97xjquMptCz3hUQ9Xtlqfvf//7jgx70oAlED3/4w8dHPvKR46Me9agp9HmgFOmvAoCRxQQkQmDTd/p/gdEzFcAQmAQOQtlH3wtwnsH0jmc4MpXKm7KNg0plSOAgKFGAiwxGkOF418sXoPJS5CVK4NJvyGCEA1K/V8ZRtnFgqAwJHISeARgZTL9XhlLku16+AJXKkjIOoWd9r/JGBgNg+uuA1O+VcZRtHBgqYfe6172mrKO/etb3gIsMRvCuAKjfk6ke8pCHrED16Ec/enzMYx4zhT7PAktgcmB5tuoFlr7bFbD0mxawvAQCrOQ4mwJLWWkpsPSX0pklcA5YykQJLKIELM9WApDAUQOWAFQDVpbAJrCUeQgHFYDycgi3SsJOKVTJ8zJIKXRQUQoJBxelUBxJJU0A8VIIoLwU6jeUQgJulYSdUijwAC6iBCovh6VSKM6kciaAAC4n0g4qAEU51F+4VRJ2SqHAQykEUJmtKIUEvMxLoXiTypmyEeASmACUl0KBKEsh4HLC7qXwEY94xF5KoWIQpyK8/HFKFFkXaYe46/vSadB5lgBGJKAg8ASnSAeXTnSAC64kkJChElCcEgk9824NXAKIgEQ4oCDwOjnqHT9FetbTaRBwUdYEEsIJPKdE/Z7QM+/WwCUACSQCE9yqROAJPet7sh6nQcBF9hFIiASUfguJ119OkS1wibQrexEDJY4MlbLDgQ984EliUOizvq9JDQIIGYhwQHEaPMpRjjK9o9BnToaUVKQGgYSToZe6BBSyA6HnltRABqLEeXYCTGc/+9mn3xN65mQIuJAaBBJOhgIJ4YDiNKjfK9vpLyfDmtTA6c5LnQNKpVOyg35P6Jl3S+DyDETJc0C57EAgU9TAxclQpZEY/NTnEoNA5DqW/tZ0LJVRBwinvdSx9Bt0LAKOlyXVwUUGIlxi0G+cy9V0LGU7lVIHSOpYAlVJx4LjlUpqScei3JGhABRcjtAz7wIulU5xMwcIZS51LAGppmNlSa3pWGhZSAwCknM5OF6CCyJ/3/vedwIXOpYy2KRjCSAEZY8s5eIoIbDp/9GzVDrJWgKPlzfXrchScDkCcJUOAkgPZC5CQNP3ykwOKr3jBwgOAspwnrUEHoFIYCI8UzmonN/VDgLKQAIP5c1BJdCQpeByhJ55Fz3Ls5YyEgKpg0rf6/9F+OFyhJ5LBwH4lsBDeSP0rO8FHgeV8zveRc9KgdTBpb//32QsAWv/jLUHMxb8yk+BCTD4VY1jCTACzxzHcoARqO+8m6BywVOEvaa6O8/SZ7KccyyBykm4wEGUVHd4lv6ivvOug6rFsVDdnWMRZLnkWIAKEi5wuF7lpL3GsXjXQVXiWK66O8DgWfpLlkuOBagEpr1xLMkLqbrnyZBActD/67cpkur0J2CgvtekBjIZ2Y13U8sSoHQCFD9SSSJcv3LV3e8UkRzgVl7CpK4LGKjvqO6lkyEBoPw6x0sYJ0JKEqq7Sw1kMcKvgshUXsJ0AkRyIEpSA5nM1Xe/zvESxolQGYdw/Qoi76USMOr0qN+nlqUTIZKDQp8Hzx4IpSmSEnoWoPwk6Hd8cBuyBhnERVKXIBSIo5mlHBRkDULPKZIiQXCfqO/9JOh3fHAbzx4IpQCNO0TCFXuXGRwU4jZkDThQSSQl9Mw1EOXLQQG3UdYg/JI4JQiXJlJmABQIo+JGhJ5TJNXvCVR3PwkqQwlQiKOPfexjx8c97nFT6PPgbgSXBNKJANFOJ0ISbbKGE+2SEwGSredeop36UoKs5UTIOz5KUI8TYR2iTRZBEnAnAiQbMNaIdupI6URwLkT0EG13IuizgKLvE2QEYEzFXUKoQPT4xz9+fOITnzg+6UlPWsWwqeaEoOluhHU1pwSVOwpKmhN8qEdzQtB0N0Kv5kR283d3pTkJHE6UlW3gREs0pxKoapqTnvW9AwyQ6a+4VAlUyk5PeMITxic/+cnjU57ylPGpT33qKgZKkt/hZfZyiwskndObcyLUbkoSRNvBBdEmyHJZ/iDagMrLkrsPyF6QbYg2JJ3Tm3Mi1G4vSw6utLcgZJbKn8Ah4PglcF4OZ/YiyHJ+enNOhFLuZQke5DIBZBtQlcqfwKHsJKItcEgtJ/TMaQ5w6fcOKt7lCkd3gspUyk4C1dOe9rQpnv70p09/p4zloIJcr3OC84y1jROcn8LyBCdgbeME56By39Q2TnCKnhNcyZC3J05wgErP+l7gcVARvEvGErDEq5SxVAKVsZSlANeUsfxCuMWzcD9QDktZC4BwKvPTWgqaBGSfd0tZC34lMNWuXtCd0JwcmJm1AIiARHgZdDLvLggHZmYt15zytJY8i9CzAzOzFgBR9vHTWknQJNCceDezFgARkIgsg+56gF85MD1rAS5lLgGMGPxC2AGV5j8FPqyWoIlSTiSgkCAIV8rhWnkiJANl2QNMSBAEVps5QdMvhEsKOTIE0gQEHq7l4ILA+4WwA8rdDgRWm5agiYPTnZ0OKCQIwk+T8DQHlyvlWfYAExKEmwJrrgYyl8qi+BYxQM

The above invocation differs from earlier: the `Inplaces.binary1` method requests a `math.add` binary op which mutates its first argument inplace. A BinaryInplace1Op instance is returned, upon which we then call mutate(). In order to show that the content of the mutated image actually changes, we also calculate and print the maximum pixel value of the image using the `stats.max` op. If the above cell is executed more than once, one can see that the maximum of the image actually changes as values are added to the pixels. Note that we stored the `INPLACE` op in a variable. Such ops can be used in other ways too; see `REUSING SPECIAL OPS` below.


## UNARY and NULLARY OPS
In addition to binary ops which have the `ARITY` 2, there are also `UNARY` (one primary input, one primary output) and `NULLARY` (no primary inputs, one primary output). Next, we are going to look at an example for both. First, we look at the `UNARY` `math.sqrt` op which simply calculates the square root of a number. Note, that this op is also a `COMPUTER`, meaning we first create an output container to write the result to.

In [7]:
import net.imglib2.type.numeric.real.DoubleType

// Create preallocated output
number = new DoubleType();
// Calculate the square root of 64
ij.op().run("math.sqrt", number, new DoubleType(64))

"The sqrt of " + 64 + " is " + number

The sqrt of 64 is 8.0

Second, we look at the `NULLARY` `math.zero` op. This op simply sets a number to zero. Note, that the variable `number` still contains the square root of the calculation from the cell above.

In [8]:
import net.imglib2.type.numeric.real.DoubleType

ij.op().run("math.zero", number)

"number == 0.0? : " + number.equals(new DoubleType(0))

number == 0.0? : true

## Reusing Special Ops
One of the most useful characteristics of special ops is the ability to look them up in a type-safe way, obtaining a reference to an op instance which can then be used however you need -- one time or many.

In [9]:
import net.imagej.ops.special.computer.Computers
import net.imagej.ops.Ops

// Create output for the COMPUTER
computerOutput = ij.op().run("create.img", sinusoid)
// Get an instance of the "math.add" op and store it for later
addOp = Computers.binary(ij.op(), Ops.Math.Add.class, computerOutput, gradient, cross)

net.imagej.ops.math.IIToIIOutputII$Add@3ce19b95

In this case, we use the `Computers.binary` utility method to require a `BINARY` `COMPUTER` `Op`, so that we can take advantage of the BinaryComputerOp API, which is type-safe -- whereas the Op interface's `run("op.name", inputs..)` method is not.

As usual, compile-time type safety is a two-edged sword: helpful to avoid coding errors, but sometimes rather verbose. As this notebook uses Groovy, which is a Scripting language, we can get around some of Java's verboseness. In java, the line from above creating the op would look like this:

```java
final BinaryComputerOp<IterableInterval<DoubleType>, IterableInterval<DoubleType>, IterableInterval<DoubleType>> addOp = Computers.binary(ops, Ops.Math.Add.class, computerOutput, gradient, cross);
```
However, the verbose way of doing it helps to understand what it going on underneath.

In the cell below we can now just call the `run()` method to execute the op using the inputs specified above.

In [10]:
addOp.run()
ij.notebook().display(computerOutput)

<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAJYAAACWCAYAAAA8AXHiAAAFZ0lEQVR42u3ScWcQDBSF8TtZkk1mmWVmSZZpJk2mSZaUTElKykxJlpRkSZpkk2WmJCVLUpKSkmySJikpSZNssiRlUlIyy2TJzut+gnv/fXkOzxc4fmZmonwFBQWaMWOGCgsLNXPmTM2aNUuzZ89WUVGRiouLNWfOHJWUlKi0tFRz585VWVmZysvLNW/ePFVUVKiyslJVVVWaP3++FixYoIULF6q6ulqLFi1STU2NFi9erNraWtXV1WnJkiVaunSp6uvrtWzZMjU0NGj58uVqbGzUihUrtHLlSjU1NWnVqlVavXq11qxZo7Vr12rdunVqbm7W+vXrtWHDBm3cuFGbNm3S5s2btWXLFm3dulXbtm3T9u3b1dLSotbWVu3YsUM7d+7Url27tHv3brW1tWnPnj3au3ev9u3bp/379+vAgQM6ePCg2tvbdejQIR0+fFhHjhzR0aNH1dHRoWPHjun48ePq7Oz0r8ACLGABC1jAAhYBC1jAAhawgEXAAhawgAUsYBGwgAUsYAELWAQsYAELWMACFgELWMACFrCARcACFrCABSxgEbCABSxgAQtYBCxgAQtYwAIWAQtYwAIWsIBFwAIWsIAFLGARsIAFLGABC1gELGABC1jAAhYBC1jAAhawgEXAAhawgAUsYAELWMACFrCABSxgAQtYwAIWsIAFLGABC1jAAhawgAUsYAELWMACFrCABSxgAQtYwAIWsIAFLGABC1jAAhawgAUsYAELWMACFrCABSxgAQtYwAIWsIAFLGABC1jAAhaw/pew/CjKB6oYVVdXl8yPonygilGdOHFC5kdRPlDFqLq7u2V+FOUDVYzq5MmTMj+K8oEqRtXT0yPzoygfqGJUvb29Mj+K8oEqRnXq1CmZH0X5QBWjOn36tMyPonygilGdOXNG5kdRPlDFqM6ePSvzoygfqGJU586dk/lRlA9UMarz58/L/CjKB6oY1YULF2R+FOUDVYyqr69P5kdRPlDFqC5evCjzoygfqGJUly5dkvlRlA9UMarLly/L/CjKB6oY1ZUrV2R+FOUDVYzq6tWrMj+K8oEqRnXt2jWZH0X5QBWjun79usyPonygilHduHFD5kdRPlDFqG7evCnzoygfqGJUt27dkvlRlA9UMarbt2/L/CjKB6oY1Z07d2R+FOUDVYzq7t27Mj+K8oEqRnXv3j2ZH0X5QBWj6u/vl/lRlA9UMaqBgQGZH0X5QBWjun//vsyPonygilE9ePBA5kdRPlDFqB4+fCjzoygfqGJUg4ODMj+K8oEqRvXo0SOZH0X5QBWjevz4scyPonygilE9efJE5kdRPlDFqJ4+fSrzoygfqGJUz549k/lRlA9UMarnz5/L/CjKB6oY1YsXL2R+FOUDVYzq5cuXMj+K8oEqRvXq1SuZH0X5QBWjev36tcyPonygilENDQ3J/CjKB6oY1Zs3b2R+FOUDVYzq7du3Mj+K8oEqRjU8PCzzoygfqGJUIyMjMj+K8oEqRvXu3TuZH0X5QBWjGh0dlflRlA9UMar379/L/CjKB6oY1YcPH2R+FOUDVYzq48ePMj+K8oEqRvXp0yeZH0X5QBWj+vz5s8yPonygilGNjY3J/CjKB6oY1ZcvX2R+FOUDVYzq69evMj+K8oEqRvXt2zeZH0X5QBWj+v79u8yPonygilH9+PFD5kdRPlDFqH7+/CnzoygfqGJUv379kvlRlA9UMarx8XGZH0X5QBWjmpiYkPlRlA9UMarfv3/L/CjKB6oY1eTkpMyPonygilH9+fNH5kdRPlDFqKampmR+FOUDVYzq79+/Mj+K8oEqRvXv3z+ZH0X5QBWjmp6elvlRlA9UMSrff/WKILnWHcIrAAAAAElFTkSuQmCC" />

Or we just run the op specifying different arguments by calling the `compute()` method. Note that for the `compute()` method the order of arguments will change. In this case the output to write to will be the last argument. Also note that the name of the method coincides with the type of op we are using. Here, we are using a `COMPUTER` op, hence the method is called `compute()`. In the case of an `INPLACE` op, the method will be called `mutate()` as one of the inputs is mutated inplace.

In [11]:
// Call the op using different inputs
addOp.compute(gradient, spot, computerOutput)
ij.notebook().display(computerOutput)

<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAJYAAACWCAYAAAA8AXHiAAAahklEQVR42u3dg5NtxxYG8J0X27b5Ytu2bdu2bdu2k5f3Ytu2jUoqqaSS9apPqueu6bvY3efcmcncqv0P3POrtb79dc85TdM0gJ/BBhus2/Ovf/2r6xl88MG7PUMMMUTXM+SQQ3Y9Qw01VOsZeuihu55hhhmm9Qw77LBdz3DDDdd6hh9++NYzwggjdD0jjjhi6xlppJFaz8gjj9x6RhllFBh11FFbz2ijjdZ6Rh999NYzxhhjwJhjjtl6xhprrNYz9thjwzjjjNN6xh13XBhvvPFaz/jjjw8TTDABTDjhhK1nookmgoknnhgmmWSS1jPppJPCZJNNBpNPPjlMMcUUMOWUU7aeqaaaCqaeemqYZpppYNppp4XpppsOpp9+ephhhhng3//+N8w444ww00wzwcwzzwyzzDILzDrrrDDbbLPB7LPPDnPMMQfMOeecMNdcc8Hcc88N88wzD8w777ww33zzwfzzzw8LLLAALLTQQrDwwgvDIossAosuuigstthisPjii8MSSywBSy21FCy99NKwzDLLwLLLLgvLL788rLDCCrDiiivCSiutBKussgqsuuqqsNpqq8Hqq68Oa665Jqy11lqw9tprw7rrrgvrrbcerL/++rDhhhvCRhttBBtvvDFsuummsNlmm8Hmm28OW265JWy11VawzTbbwLbbbgvbbbcd7LDDDrDjjjvCzjvvDLvssgvsuuuusPvuu8Mee+wBe+21F+y9996w7777wn777QcHHHAAHHjggXDwwQcHSzZUKSwNVQmsFFWEFVD9U2EtueSSLlhrrLGGC9YWW2wxEKztt99+IFi77bZbC9aee+5pg1UDlTatMKyIygILT6t+WH/DWm655brBWnnllVlY66yzThesDTbYoAvWJptsMhCsrbfeeiBYO+20Ewtrn332acHaf//9W7AOOuigAbA8qNo1rdoBK6CiYAVU/bAGwAprMIUV1mAKK6zBACusQRVWp1DVylccrICqt8NacMEFSVghX3lghXwVYYU1GGGFNRhhhTWYwgprMIUV1iAHK6zBACuswQArrMEA65BDDukOy4OqBJZlDdaGFVD1FVghX2FYIV/lworBHcOKwR3DSoO7GVanUHnXoAYLr8F+WKt0vRFKsGJwx7BicMewYnCnYEnBvQtWLVS508qSr1JYVL7qzbBCvqJgxaohB1Z8Iwyw4hshhuWtGmJwN8EqQVUyrWrmq38SrFg1RFixarDCsnZYlqqBC+6HHnpod1i1UJWuwVJYuGrozbBwOWqBlVuOWqoGzxthN1glqEqmVUm+6iuwYtXghYXLUS8sqcOyVA1ScG/BSkHVROWZVv2wdFi4dU9htaPD0oI7B+uwww4bGFYJqhrTyhLcPa17b4WVnhPWhCV1WJaqQQvu3WCloGqi8oR2S77qS7Bw6+6BZTknLOmwSoJ7F6zaqLzTquYalM4JezMs6QDaU45KHZalajDD0kB1ElXJGiyBFVBhWAHVoISlXZnxHud4O6zS4H744YcPgGUB5UXVD0uGlR5Ac7Ckc8LSDqv2G2GYVl2wehoqag32w8qDlVs1lAT3FqwcUCWoSqdVTutO3cXqqbC0KzOW4xyqw5Kqhtw3Qi5fHXHEEd1htQNVybSS1mBvh0XdxaJgSeeEWjmaWzWUBPcwrbpg5YIqQdWJNajdHu2LsKgOS6oarGeEnuDegmUBVQNVjWnV12FpV2Y85Whu1ZAT3NN8deSRRw6AVQKqBJVnWvUVWNZrydZzwtyqoR3BPUyrFiwPqBqovNMqJ1/1RVi55Wi73gil4E7CKgFVgipnDVph4aqhp8LyHEBrsCxVQ83gnuaro446agAsDyjLlNJQeaeVtAZ7EyzrtWQLLGuHVfpG6ClGw7RqwSoF1WlUljUolaO9EZanHLVUDTWCO1eMBlhHH310d1gcphJQGqrcaeXNVz0ZlucAmoIldVjeN0JPcOfyVQuWF9OgRNXbYVmvJVtad6nDkqqGnODuKUbDGmRheTBZQXlQedagFxZu3fsCLKnDakdwl4rRmK+OOeaYAbC804kDlYOqxrTqrbCsB9DWDku61eCBlVuMhmnVguXFVArKgypnWvVUWNZryZ7W3VI11AjuUjFK5atjjz22OywJkxdUDirvtJLWoOUAelDD8pwTSh2W5VZDaXD35KsWrFxMpaA8qGqtQelack+DldNhaW+EnuAuFaNaviJhSZAkTBwoy5SyrkBuWvV0WNZryRosqWqQbjVQsEqLUSlfHXfccdBokDRMHlClqLRpVQIroOokLOmcUOuwLFWDJ7jXyFd4DbKwNEgSphJQ1hWoTau+CEuqGrQ3Qk9wtx48U+eDcQ0ef/zxf8MqhcRh6iQqalr1BFiWa8nccQ7VYVmqBktw9+Yry/lgXIMtWLmQcjBZQUmoctag1LoPCljSOaFWjmpvhDnB3ZqvpPtXeA2ecMIJf8OyILJg8oKyoPJOK+sa7E2wPFWDBEsqRq35ijofxPkqTKsWrBJEGqZSUBKq0jXYCViWa8meclR7I/QEd2u+0u5fpTVDgHXiiScOgGVFpEGSMHGgtCmVM616KizpnJCC5akaMCxLMVorX+GaIUyrFqxSRBZMHlBeVNoa7MmwcjqsnOBeO19JNUOAddJJJ/0Ny4rHikjDxIHSgnrJtLIc59SEZbmWrMHyVg25xaiWr7RjnHQNtmDVQqRBkjBZppSGyroGByUs6QDa0mF5gnvtfGVZgxHWySefDI0HjgdRLiYLKCsq6xocFLC0crRmcM/JV9oxDlczhGmlwrLisULKAeVF5VmDnYQlnRN6qgYJllSMavmKun/F1QzaGjzllFOgycFjBaRB8oLyoBqUsLRLflqHVSO45+Sr3JohXYMiLA8cDyINEweqBioNVvptfp2EZakaKFhSMarlK+1+e+4aPPXUU6FpNyALJC8oDZU2rToNizsnpDqsnOBuzVfU+SBXM5SswdNOO21gWF4wOYA0SBKoXFSdgqVd8qPKUUvVkFOMUv1Vbs3gWYMtWO2E40GkYaJAaag8a7DTsKSqwRLcrflKu3/VjjV4+umnQ5MLpRSQBRKHqQRVp2Fp5ahWNXjzFdVfcTWDt223rkEzLC+WXEQWTBQoDVWnYGmX/CxVgxTcrfkqt2bIWYPxbDBOqwDrjDPOgKbTaHIgcZgoUFZUnYCllaM5wd3aX3E1QyfW4JlnntkdVgmOGngskCRQGipqWvV0WNZ8xdUM2p95cW177ttgmFYtWJ2CUoJIw0SBklC1A5Z2yY+rGizBPSdfcTVDzhrUrsika/Css86CpiaQEjReSBIoDRW1BqnWvQSWVo5KsHLyVSfWIHVTNE4rvAZZWCUY2oHHAokDJaHiplW7YHFvhFRw1/KVp2aQ1mBOKcp1V3ENnn322dAMKiA1EEmYPKhKYUkH0FKHVZqvaq5B7xUZKrTHNXjOOedA0w4QtcDkQOJApag6DcsS3LV81Yk1aLkpynVXcVqde+653WGVfuA1HwseCyYPqtqwuDdCa76izge5mkFagzlngzndVZxWLVg9AURNQBImClUpLOkA2vJGqOWrmmvQckVG666k0B6mVYB13nnnQdMJCLWg5ELiQHGo8F2sXFie4I7zVbvWYI3uSgrtYVrFNXj++edDU+tD7cSjwfGASlFx08oDi+qwKFg4X2nng9Y1aD0blLorKbRrTXtcg2FaFcPK/aDb9XCANFDUtLLA4g6gLcEd56vSNcidDaZXZHJDu6Vpj9MqwLrgggug6Y0wch4KE4UqFxbVYXH5iluDuGbIWYNSdyWFdunAWWra04ohrsELL7wQmnZ+mJ14JDDak4LCqHJhUcFd66+kNciVounZYG5oz23a04ohrsEwrS666CJoSj6Y3vJQgDRUGiyudefyVekaxGeD6R

An op may additionally have secondary inputs. These are usually algorithm parameters or an `ENUM` that specifies the behaviour of the op. One example would be the sigma value or the out-of-bounds strategy used for a Gaussian filter op, which would look like this (sigma = 2, out-of-bounds = Boundary.SINGLE):

```java
gaussSigma2ComputerOp = Computers.unary(ij.op(), Ops.Filter.Gauss.class, input, input, 2, new OutOfBoundsMirrorFactory(Boundary.SINGLE))
```
Note the difference between primary and secondary inputs: the latter are not given when calling the special op's type-safe API; hence, secondary inputs remain fixed as given during the original matching (when we requested the op). Thus, the `UNARY` `COMPUTER` we saved in the variable `gaussSigma2ComputerOp` will always perform a Gaussian filtering with sigma 2. Example usage could look like this:

```java
gaussSigma2ComputerOp.compute(image, smoothedImage)
```

The variable `smoothedImage` will now contain the smoothed image using a sigma value of 2 as defined above when we requested the op.

Which inputs are considered "primary" is a decision left to the author of each op. But here are two rules of thumb:
* The primary inputs and outputs should correspond to the main relevant data structure -- e.g., ops which operate on images should declare the image argument(s) as primary.
* There is usually an anticipated main use case involving variation of specific input(s) -- those should be chosen as the main inputs, so that the op can be reused across different values of those inputs. Hence, the inputs that change when executing the op several times are the primary ones, i.e. in most cases the input/output images.

### Reuse an op to improve performance
It is important to understand that each call into the Ops framework requires the system to search for the best-matching op amongst all those available. This is very useful for reasons of extensibility: the best op for the job is always selected, and anyone can extend the system with new ops optimized for particular scenarios. However, matching can have substantial performance implications:

In [12]:
import net.imglib2.type.numeric.real.DoubleType
import net.imglib2.IterableInterval

slowStart = System.currentTimeMillis();
200.times {
    // Search for the op every time and then run it
    ij.op().run("math.add", computerOutput, gradient, spot)
}
slowEnd = System.currentTimeMillis();
println("ij.op().run() needs\t" + (slowEnd - slowStart) + " ms for 200 iterations")

fastStart = System.currentTimeMillis();
200.times {
    // Execute the same op instance repeatedly
    addOp.compute(gradient, spot, computerOutput)
}
fastEnd = System.currentTimeMillis();

println("addOp.compute() needs \t" + (fastEnd - fastStart) + " ms for 200 iterations")
""

ij.op().run() needs	2812 ms for 200 iterations
addOp.compute() needs 	14 ms for 200 iterations


As one can see, reusing the same matched op every time is much faster. However, it is the responsibility of the caller to ensure that subsequent arguments passed in are compatible with that same op.

### Ops Chaining
Some ops accept other ops as arguments, forming chains or trees of ops. For example, the map op executes the given op over all elements of an iteration. In the following example, we want to take the pixelwise square root of an image using the `math.sqrt` op. Unfortunately, this op is only applicable for singe numbers compared to the `math.add` op which can already handle `IterableInterval` for convenience. However, we can make use of the `map` op which expects another op as input and executes it on every element of an iterable; in this case every pixel. 

In [13]:
import net.imagej.ops.special.computer.Computers
import net.imglib2.type.numeric.real.DoubleType
import net.imagej.ops.Ops

// Create output for the COMPUTER
mapOutput = ij.op().run("create.img", sinusoid)

imgMax = ij.op().run("stats.max", sinusoid)
println("The maximum pixel value:")
println("* before pointwise square root is:\t\t" + imgMax)
println("* after pointwise square root should be:\t" + ij.op().run("math.sqrt", imgMax.get()))

// Create and save a sqrt op operating on DoubleType
sqrtOp = Computers.unary(ij.op(), Ops.Math.Sqrt.class, DoubleType.class, DoubleType.class)

// 'Map' the op on the 'sinusoid' image to calculate the square root of every pixel.
ij.op().run("map", mapOutput, sinusoid, sqrtOp);

println("* after pointwise square root is:\t\t" + ij.op().stats().max(mapOutput))
""

The maximum pixel value:
* before pointwise square root is:		19.999118601072674
* after pointwise square root should be:	4.472037410518015
* after pointwise square root is:		NaN


In the above example, we additionally calculate some image statistics, namely the maximum pixel value in order to verify that the `map` op is actually doing what we'd expect. First we print the maximum of the `sinusoid` image and the square root of that value. After the map operation, the new maximum of the map output should be the same as previously calculated one, which is indeed the case :)

Next, we want to look at a slightly more advanced example of chaining. Here, we want to calculate the Difference of Gaussian of an image which is a classical image processing operation. If we look at the method signatures of the `dog` op, we see that there is a version with the following inputs:
* RandomAccessibleInterval out? : the output to write to
* RandomAccessibleInterval in : the input to calculate the dog on
* UnaryComputerOp gauss1 : an op that calculates the first Gaussian filtered version of the image
* UnaryComputerOp gauss2 : an op that calculates the second Gaussian filtered version of the image
* UnaryFunctionOp outputCreator : an op to allocate output if the op is used as a FUNCTION
* UnaryFunctionOp tmpCreator : an op to create a temporary storage needed by the algorithm in order to calculate the first Gaussian

Therefore, we will create all of these ops and call the `dog` op using them:

In [14]:
import net.imglib2.type.numeric.real.DoubleType
import net.imagej.ops.special.computer.Computers
import net.imagej.ops.Ops;
import net.imglib2.RandomAccessibleInterval
import java.util.Collections;
import net.imagej.ops.special.function.Functions;
import net.imglib2.outofbounds.OutOfBoundsMirrorFactory;
import net.imglib2.outofbounds.OutOfBoundsMirrorFactory.Boundary;

// For this example we will use a different image as this will illustrate the result of the `dog` more nicely
input = ij.scifio().datasetIO().open("https://imagej.net/images/clown.png")
// Convert the input to float in order to get a corerct result
input = ij.op().run("convert.float32", input)

Class raiClass = RandomAccessibleInterval.class
// Create two Gaussian filter ops with different sigmas as secondary inputs. Furthermore, we need to specify an out-of-bounds strategy
gaussSigma2Op = Computers.unary(ij.op(), Ops.Filter.Gauss.class, input, input, 2, new OutOfBoundsMirrorFactory(Boundary.SINGLE))
gaussSigma8Op = Computers.unary(ij.op(), Ops.Filter.Gauss.class, input, input, 8, new OutOfBoundsMirrorFactory(Boundary.SINGLE))

// Create the tmp and output creator ops
outputCreatorFunc = Functions.unary(ij.op(), Ops.Create.Img.class, raiClass, input)
tmpCreatorFunc = Functions.unary(ij.op(), Ops.Create.Img.class, raiClass, input)

// Allocate output
dogOutput = ij.op().run("create.img", input)
// Run the `dog` op
ij.op().run("dog", dogOutput, input, gaussSigma2Op, gaussSigma8Op, outputCreatorFunc, tmpCreatorFunc)

ij.notebook().display(dogOutput)